In [1]:
import operator
from functools import partial

In [2]:
with open('inputs/day-05.txt', 'r') as f:
    code = f.read()[:-1]  # drop eof newline
print(code[:79])

3,225,1,225,6,6,1100,1,238,225,104,0,1102,46,47,225,2,122,130,224,101,-1998,224


## machine

In [3]:
def resolve(mem, val, mode):
    return mem[val] if int(mode) == 0 else val


def op(op_func, num_params, mem, pointer, modes, **kwargs):
    params = mem[pointer + 1:pointer + num_params + 1]
    if kwargs.get('_debug', False):
        print("", params)
    p_orig = pointer

    mem, pointer, *state = op_func(mem, pointer, params, modes, **kwargs)

    state = state[0] if state else "running"
    pointer = (pointer + len(params) + 1) if (pointer == p_orig) else pointer
    return mem, pointer, state


def opcode(instruction):
    return int(str(instruction)[-2:])


def modes(instruction):
    return f"{instruction:05d}"[:3][::-1]


def run(ops, mem, pointer, state, **kwargs):
    if state == "halted":
        return mem

    instruction = mem[pointer]
    _opcode = opcode(instruction)
    _modes = modes(instruction)
    if kwargs.get('_debug', False):
        print(_opcode, _modes, end='')

    _op = ops[_opcode]
    return run(ops, *_op(mem, pointer, _modes, **kwargs), **kwargs)


#### operations

In [4]:
# f(mem: List[int], pointer: int, params: List[int], modes: str, **kwargs)
#     -> Tuple[List[int], int, Optional[str]]

def _math(func, mem, pointer, params, modes, **kwargs):
    a = resolve(mem, params[0], modes[0])
    b = resolve(mem, params[1], modes[1])
    out = params[2]
    res = int(func(a, b))
    _mem = list(mem)
    _mem[out] = res
    return _mem, pointer


def _inp(mem, pointer, params, modes, **kwargs):
    inp = kwargs.get('_input', None)
    if inp is None:
        inp = input("inp: ")
    out = params[0]
    _mem = list(mem)
    _mem[out] = int(inp)
    return _mem, pointer


def _out(mem, pointer, params, modes, **kwargs):
    val = resolve(mem, params[0], modes[0])
    print("out:", val)
    return mem, pointer


def _jump(func, mem, pointer, params, modes, **kwargs):
    cond = resolve(mem, params[0], modes[0])
    _pointer = pointer
    if func(cond):
        _pointer = resolve(mem, params[1], modes[1])
    return mem, _pointer


def _halt(mem, pointer, *args, **kwargs):
    return mem, pointer, "halted"

## part 1

In [5]:
# partial(_op, op_func, num_params)
OPS_PT1 = {
    1: partial(op, partial(_math, operator.add), 3),
    2: partial(op, partial(_math, operator.mul), 3),
    3: partial(op, _inp, 1),
    4: partial(op, _out, 1),
    99: partial(op, _halt, 0)
}

def part_1(mem, **kwargs):
    return run(OPS_PT1, mem, 0, "running", **kwargs)


mem = list(map(int, code.split(',')))
part_1(mem, _debug=False);

inp: 1
out: 0
out: 0
out: 0
out: 0
out: 0
out: 0
out: 0
out: 0
out: 0
out: 12896948


## part 2

In [6]:
# partial(_op, op_func, num_params)
OPS_PT2 = {
    **OPS_PT1,
    5: partial(op, partial(_jump, lambda x: x != 0), 2),
    6: partial(op, partial(_jump, lambda x: x == 0), 2),
    7: partial(op, partial(_math, operator.lt), 3),
    8: partial(op, partial(_math, operator.eq), 3),
}

def part_2(mem, **kwargs):
    return run(OPS_PT2, mem, 0, "running", **kwargs)


mem = list(map(int, code.split(',')))
part_2(mem, _debug=False);

inp: 5
out: 7704130
